In [ ]:
import pandas as pd
import json
import os
import re

# === 配置区域 ===
CSV_FILE = 'works.csv'  # 你的表格文件名
OUTPUT_FILE = 'data.js' # 输出的js文件名

# 固定的静态数据
STATIC_DATA_START = """/* 自动生成的 data.js */
const siteData = {
    backgrounds: [
        "background/bg1.JPG", 
        "background/bg2.JPG", 
        "background/bg3.JPG",
        "background/bg4.JPG", 
        "background/bg5.JPG", 
        "background/bg6.JPG",
        "background/bg7.JPG", 
        "background/bg8.JPG", 
        "background/bg9.JPG",
        "background/bg10.JPG", 
        "background/bg11.JPG", 
        "background/bg12.JPG",
        "background/bg13.JPG"
    ],
    news: [
        {
            title: "TEN BY ONE",
            location: "PEAC Museum Freiburg",
            time: "21.09.2025 - 08.02.2026",
            details: "Opening 21.09.2025, 11h. Full info line 1\\nLine 2"
        }
    ],
    about: "1000-word introduction... Here is the text about the artist.",
    contact: "Email: zihan@example.com<br>Instagram: @zihanteng",
    
    // 下面是自动生成的作品列表
    works: """

STATIC_DATA_END = "\n};"

def natural_sort_key(path):
    """
    排序核心逻辑：提取文件名末尾的数字进行排序
    例如: 'luoye2.jpg' -> 2, 'luoye10.jpg' -> 10
    这样 2 就会排在 10 前面，而不是后面
    """
    filename = os.path.basename(path)
    name_without_ext = os.path.splitext(filename)[0]
    
    # 查找文件名里的所有数字
    numbers = re.findall(r'\d+', name_without_ext)
    
    if numbers:
        # 取最后一个数字作为排序依据 (例如 'img_2025_01.jpg' -> 1)
        return int(numbers[-1])
    else:
        # 如果文件名没数字，排在最前面
        return 0

def clean_image_path(raw_path):
    """
    清洗并排序图片路径
    """
    if pd.isna(raw_path) or str(raw_path).strip() == '':
        return []
    
    # 1. 拆分多行路径
    paths = str(raw_path).split('\n')
    cleaned_paths = []
    
    for p in paths:
        # 获取文件名
        filename = os.path.basename(p.replace('\\', '/'))
        # 去掉引号和空格
        filename = filename.strip().strip('"').strip("'")
        
        if filename:
            # 强制改后缀为 .jpg (配合之前的图片转换脚本)
            if filename.lower().endswith('.heic') or filename.lower().endswith('.png'):
                filename = os.path.splitext(filename)[0] + ".jpg"
                
            cleaned_paths.append(f"selectedphoto/{filename}")
            
    # 2. 【新增】执行自然排序
    cleaned_paths.sort(key=natural_sort_key)
            
    return cleaned_paths

def generate_id(name):
    """生成 ID"""
    if pd.isna(name): return "unknown-work"
    # 替换特殊字符，保留字母数字和连字符
    safe_name = str(name).strip().lower()
    safe_name = re.sub(r'[^a-z0-9\s-]', '', safe_name)
    return safe_name.replace(' ', '-').replace('--', '-')

def process_csv():
    print(f"正在读取 {CSV_FILE} ...")
    try:
        df = pd.read_csv(CSV_FILE, encoding='utf-8') 
    except UnicodeDecodeError:
        print("UTF-8 读取失败，尝试 GBK 编码...")
        df = pd.read_csv(CSV_FILE, encoding='gbk')

    works_list = []

    for index, row in df.iterrows():
        # 1. 生成 ID
        work_name = row.get('work_name', '')
        if pd.isna(work_name) or str(work_name).strip() == '':
            continue # 跳过空行

        work_id = generate_id(work_name)
        
        # 2. 处理 Intro
        t = str(row.get('work_time', '')).strip()
        m = str(row.get('work_ms', '')).strip()
        o = str(row.get('work_ort', '')).strip()
        
        # 替换表格里的换行符
        m = m.replace('\n', '<br>').replace('\r', '')
        o = o.replace('\n', '<br>').replace('\r', '')
        
        # 拼接非空字段
        intro_parts = [p for p in [t, m, o] if p and p != 'nan']
        intro_html = " <br> ".join(intro_parts)

        # 3. 处理 Concept
        concept = str(row.get('work_concept', ''))
        if concept == 'nan': concept = ""
        concept_html = concept.replace('\n', '<br>').replace('\r', '')

        # 4. 处理图片 (含排序)
        images = clean_image_path(row.get('work_photo', ''))

        # 5. 构建对象
        work_obj = {
            "id": work_id,
            "name": str(work_name).strip(),
            "intro": intro_html,
            "concept": concept_html,
            "images": images
        }
        works_list.append(work_obj)
        print(f"处理作品: {work_name} -> 包含 {len(images)} 张图片")

    # 生成最终 JS 文件
    json_str = json.dumps(works_list, ensure_ascii=False, indent=4)
    
    with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
        f.write(STATIC_DATA_START)
        f.write(json_str)
        f.write(STATIC_DATA_END)
    
    print(f"\n✅ 成功生成 {OUTPUT_FILE}！")
    print("现在所有的图片都已按文件名末尾的数字顺序（1, 2... 10）排列。")

if __name__ == "__main__":
    process_csv()